# Improving Triangle counting in LightGraphs
We want to make triangle counting faster by improving the algorithm.
BenchmarkTools is used to make sure that we are measuring times with sufficient accuracy.

In [1]:
using LightGraphs
using BenchmarkTools

## Original Implementation

In [2]:
function local_clustering(g::AbstractGraph, v::Integer)
    k = degree(g, v)
    k <= 1 && return (0, 0)
    neighs = neighbors(g, v)
    c = 0
    for i in neighs, j in neighs
        i == j && continue
        if has_edge(g, i, j)
            c += 1
        end
    end
    return is_directed(g) ? (c , k*(k-1)) : (div(c,2) , div(k*(k-1),2))
end
function local_clustering(g::AbstractGraph, vs = vertices(g))
    ntriang = zeros(Int, length(vs))
    nalltriang = zeros(Int, length(vs))
    i = 0
    for (i, v) in enumerate(vs)
        ntriang[i], nalltriang[i] = local_clustering(g, v)
    end
    return ntriang, nalltriang
end


local_clustering (generic function with 3 methods)

## Using Memory to Save Time
The problem with the original implementation is that $degree(g, v)$ calls to $has_edge$ which is a $log(deg)$ operation.

If we use a two pass algorithm to precompute a constant time lookup table.
The downside is that this lookup table requires $nv(g)$ storage.

In [10]:
function local_clustering!(storage::AbstractVector{Int}, g::AbstractGraph, v::Integer)
    k = degree(g, v)
    k <= 1 && return (0, 0)
    neighs = neighbors(g, v)
    tcount = 0
    for i in neighs
        storage[i] = v
    end
    
    for i in neighs
        for j in neighbors(g, i)
            i == j && continue
            if storage[j] == v
                tcount += 1
            end
        end
    end
    
    return is_directed(g) ? (tcount , k*(k-1)) : (div(tcount,2) , div(k*(k-1),2))
end

function local_clustering!(storage::AbstractVector, g::AbstractGraph, vs = vertices(g))
    ntriang = zeros(Int, length(vs))
    nalltriang = zeros(Int, length(vs))
    i = 0
    for (i, v) in enumerate(vs)
        ntriang[i], nalltriang[i] = local_clustering!(storage, g, v)
    end
    return ntriang, nalltriang
end

local_clustering! (generic function with 6 methods)

## Can we save Memory by using bools?

We can cut the memory footprint by a factor of 8 by using a Bool Array.
But then we have to reset it after every vertex.

In [11]:
function local_clustering!(storage::AbstractVector{Bool}, g::AbstractGraph, v::Integer)
    k = degree(g, v)
    k <= 1 && return (0, 0)
    neighs = neighbors(g, v)
    tcount = 0
    for i in neighs
        storage[i] = true
    end
    
    for i in neighs
        for j in neighbors(g, i)
            i == j && continue
            if storage[j]
                tcount += 1
            end
        end
    end
    
    return is_directed(g) ? (tcount , k*(k-1)) : (div(tcount,2) , div(k*(k-1),2))
end
function local_clustering!(storage::AbstractVector{Bool}, g::AbstractGraph, vs = vertices(g))
    ntriang = zeros(Int, length(vs))
    nalltriang = zeros(Int, length(vs))
    i = 0
    for (i, v) in enumerate(vs)
        ntriang[i], nalltriang[i] = local_clustering!(storage, g, v)
        for w in neighbors(g, v)
            storage[w] = false
        end
    end
    return ntriang, nalltriang
end

local_clustering! (generic function with 6 methods)

## Shootout
Who is faster?

In [12]:
n,k = 10000,5
g = barabasi_albert(n,k)

{10000, 49975} undirected simple Int64 graph

In [13]:
@benchmark local_clustering(g)

BenchmarkTools.Trial: 
  memory estimate:  156.44 KiB
  allocs estimate:  5
  --------------
  minimum time:     79.872 ms (0.00% GC)
  median time:      87.567 ms (0.00% GC)
  mean time:        97.505 ms (0.00% GC)
  maximum time:     306.803 ms (0.00% GC)
  --------------
  samples:          52
  evals/sample:     1
  time tolerance:   5.00%
  memory tolerance: 1.00%

In [14]:
storage = zeros(Int, nv(g))
@benchmark local_clustering!($storage, $g) setup=(storage=zeros(Int, nv($g)))

BenchmarkTools.Trial: 
  memory estimate:  156.44 KiB
  allocs estimate:  5
  --------------
  minimum time:     8.330 ms (0.00% GC)
  median time:      10.456 ms (0.00% GC)
  mean time:        12.099 ms (0.06% GC)
  maximum time:     82.420 ms (0.00% GC)
  --------------
  samples:          412
  evals/sample:     1
  time tolerance:   5.00%
  memory tolerance: 1.00%

In [15]:
@benchmark local_clustering!($storage, $g) setup=(storage=zeros(Bool, nv($g)))

BenchmarkTools.Trial: 
  memory estimate:  156.44 KiB
  allocs estimate:  5
  --------------
  minimum time:     8.509 ms (0.00% GC)
  median time:      9.774 ms (0.00% GC)
  mean time:        10.338 ms (0.10% GC)
  maximum time:     25.713 ms (0.00% GC)
  --------------
  samples:          482
  evals/sample:     1
  time tolerance:   5.00%
  memory tolerance: 1.00%

## Bool Array Wins!

It looks like the bool array is slightly faster even though we have to reset it. Lets use that.